In [14]:
import h5py
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.figure import Figure
import matplotlib.axes._axes as axes
sns.set()

In [15]:
# import the necessary packages
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from loader_util.preprocessing import ImageToArrayPreprocessor, \
    AspectAwarePreprocessor, MeanSubtractionPreProcessor, SimplePreProcessor
from loader_util.datasets import SimpleDatasetLoader
from loader_util.io import HDF5DatasetGenerator
from loader_util.nn.conv import FCHeadNet
from loader_util.callbacks import TrainingMonitor
##
from tensorflow.keras.layers import Conv2D, Activation, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop, SGD, Adam
from tensorflow.keras.applications import VGG16, ResNet50
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import LearningRateScheduler
from imutils import paths
import os

In [16]:
# construct the train image generator

aug = ImageDataGenerator(rotation_range=20,
                         zoom_range=0.15,
                         width_shift_range=0.2,
                         height_shift_range=0.2,
                         shear_range=0.15,
                         horizontal_flip=True,
                         fill_mode='nearest')

In [17]:
batch_size = 64
epoch_num = 100

# initialise the image preprocessors
sp = SimplePreProcessor(width=224, height=224)
iap = ImageToArrayPreprocessor()

In [18]:
# initialise the data paths

dbBase = r"/home/mhasan3/Desktop/WorkFolder/mokn_4class_data"
dbTrainPath = f"{dbBase}//train_cad1.hdf5"
dbValidPath = f"{dbBase}//validate_cad1.hdf5"

# get the no. of classes
trainFile = h5py.File(name=dbTrainPath, mode="r")
class_num = len(list(np.unique(trainFile["labels"])))
print(f"Total no. of classes in dataset: {class_num}")

Total no. of classes in dataset: 4


In [19]:
# initialise the train and valid generators
trainGen = HDF5DatasetGenerator(dbPath=dbTrainPath,
                                batchSize=batch_size,
                                preprocessors=[sp, iap],
                                classes=class_num,
                                aug=aug)

valGen = HDF5DatasetGenerator(dbPath=dbValidPath,
                              batchSize=batch_size,
                              preprocessors=[sp, iap],
                              classes=class_num)

In [20]:
# define the Learning Rate Scheduler

initial_rate = 5e-3

def poly_decay(epoch):
    max_epochs = epoch_num
    baseLR = initial_rate
    power = 1.0
    
    alpha = baseLR * (1 - (epoch / float(max_epochs))) ** power
    return alpha

In [21]:
# load the VGG16 network ensuring the head FC layer sets are left off
baseModel = VGG16(weights='imagenet', include_top=False,
                  input_tensor=Input(shape=(224, 224, 3))) # type: Model

# initialise the new head of the network, a set of FC layers followed by
# softmax
headModel = FCHeadNet.builld(baseModel, classes=class_num, D=[1024, 512, 256]) # type: Model

# place the head FC model on top of the base model - this will be the actual
# model to train
model = Model(inputs=baseModel.input, outputs=headModel)

BaseModel Out Shape:  (?, 7, 7, 512)


In [22]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [23]:
# loop over all the layers in the base model and freeze them
for layer in baseModel.layers:
    layer.trainable = False

In [24]:
# compile and optimise model
opt = SGD(lr=initial_rate)
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

# construct callbacks
path = os.path.sep.join([dbBase, f'{os.getpid()}.png'])
callbacks = [TrainingMonitor(path), LearningRateScheduler(poly_decay)]

In [25]:
# train the head for few epochs
H = model.fit_generator(
    trainGen.generator(),
    steps_per_epoch=trainGen.numImages//batch_size,
    validation_data=valGen.generator(),
    validation_steps=valGen.numImages//batch_size,
    epochs=epoch_num,
    max_queue_size=20,
    verbose=1,
    callbacks=callbacks
)

Epoch 1/100
117/117 [==============================] - 106s 906ms/step - loss: 0.5871 - acc: 0.8034 - val_loss: 0.1382 - val_acc: 0.9794
Epoch 2/100
117/117 [==============================] - 99s 843ms/step - loss: 0.1894 - acc: 0.9474 - val_loss: 0.0608 - val_acc: 0.9877
Epoch 3/100
117/117 [==============================] - 92s 787ms/step - loss: 0.1189 - acc: 0.9700 - val_loss: 0.0378 - val_acc: 0.9931
Epoch 4/100
117/117 [==============================] - 95s 812ms/step - loss: 0.0877 - acc: 0.9789 - val_loss: 0.0289 - val_acc: 0.9943
Epoch 5/100
117/117 [==============================] - 97s 829ms/step - loss: 0.0746 - acc: 0.9809 - val_loss: 0.0201 - val_acc: 0.9997
Epoch 6/100
117/117 [==============================] - 91s 777ms/step - loss: 0.0586 - acc: 0.9878 - val_loss: 0.0210 - val_acc: 0.9994
Epoch 7/100
117/117 [==============================] - 97s 833ms/step - loss: 0.0511 - acc: 0.9886 - val_loss: 0.0139 - val_acc: 0.9994
Epoch 8/100
117/117 [==========================

Epoch 38/100
117/117 [==============================] - 98s 834ms/step - loss: 0.0125 - acc: 0.9975 - val_loss: 0.0020 - val_acc: 1.0000
Epoch 39/100
117/117 [==============================] - 97s 830ms/step - loss: 0.0109 - acc: 0.9983 - val_loss: 0.0020 - val_acc: 1.0000
Epoch 40/100
117/117 [==============================] - 98s 840ms/step - loss: 0.0103 - acc: 0.9983 - val_loss: 0.0021 - val_acc: 1.0000
Epoch 41/100
117/117 [==============================] - 112s 961ms/step - loss: 0.0101 - acc: 0.9980 - val_loss: 0.0020 - val_acc: 1.0000
Epoch 42/100
117/117 [==============================] - 126s 1s/step - loss: 0.0094 - acc: 0.9987 - val_loss: 0.0018 - val_acc: 1.0000
Epoch 43/100
117/117 [==============================] - 118s 1s/step - loss: 0.0079 - acc: 0.9993 - val_loss: 0.0018 - val_acc: 1.0000
Epoch 44/100
117/117 [==============================] - 120s 1s/step - loss: 0.0099 - acc: 0.9979 - val_loss: 0.0018 - val_acc: 1.0000
Epoch 45/100
117/117 [========================

Epoch 75/100
117/117 [==============================] - 100s 855ms/step - loss: 0.0056 - acc: 0.9995 - val_loss: 0.0011 - val_acc: 1.0000
Epoch 76/100
117/117 [==============================] - 106s 907ms/step - loss: 0.0064 - acc: 0.9987 - val_loss: 0.0013 - val_acc: 1.0000
Epoch 77/100
117/117 [==============================] - 130s 1s/step - loss: 0.0068 - acc: 0.9987 - val_loss: 0.0012 - val_acc: 1.0000
Epoch 78/100
117/117 [==============================] - 125s 1s/step - loss: 0.0070 - acc: 0.9992 - val_loss: 0.0012 - val_acc: 1.0000
Epoch 79/100
117/117 [==============================] - 124s 1s/step - loss: 0.0064 - acc: 0.9992 - val_loss: 0.0011 - val_acc: 1.0000
Epoch 80/100
117/117 [==============================] - 116s 995ms/step - loss: 0.0065 - acc: 0.9991 - val_loss: 0.0012 - val_acc: 1.0000
Epoch 81/100
 61/117 [==============>...............] - ETA: 33s - loss: 0.0067 - acc: 0.9987

KeyboardInterrupt: 

In [26]:
model.save(f"{dbBase}//mokn_4class_model.pt")